In [1]:
#import libraries
#from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# import CityIq and libraries for timing
from cityiq import CityIq
import time
import json

#load .env with credentials
from dotenv import load_dotenv
load_dotenv()

import os
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
USERNAME = os.getenv("USERNAME")

set_default_credentials(
    base_url=BASE_URL,
    api_key=API_KEY
)

credentials = Credentials(USERNAME, API_KEY)

ModuleNotFoundError: No module named 'cityiq'

In [ ]:
# set time frame for use when querying for events (epoch time in milliseconds)
endTime = int(time.time())*1000 # time when demo.py is run
startTime = endTime-3600000 # startTime is 1 hour before endTime

# get CityIq token
myCIQ = CityIq("City")
myCIQ.fetchToken()

# getting assets - assets with BICYCLE events
myCIQ.fetchMetadata("assets","bicycle","eventTypes:BICYCLE")
san_diego_bicycle_sensor_metadata = myCIQ.getAssets()

In [ ]:
san_diego_bicycle_sensor_metadata_df = pd.DataFrame(san_diego_bicycle_sensor_metadata)
san_diego_bicycle_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_bicycle_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_bicycle_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_bicycle_sensor_metadata_df["longitude"]= latlng[1].astype(float)
#print(san_diego_bicycle_sensor_metadata_df)

san_diego_bicycle_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_bicycle_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_bicycle_sensor_metadata_df.longitude, san_diego_bicycle_sensor_metadata_df.latitude))
print(san_diego_bicycle_sensor_metadata_gdf)

In [ ]:
san_diego_bicycle_sensor_metadata_d = Dataset(san_diego_bicycle_sensor_metadata_gdf)
san_diego_bicycle_sensor_metadata_d.upload(table_name='san_diego_bicycle_sensor_metadata', if_exists='replace', credentials=credentials)

In [ ]:
# empty list to collect events
san_diego_bicycle_sensor_events_list = []
# loop through assets to fetch events for each asset
for index, row in san_diego_bicycle_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "BICYCLE", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        a["directionUnit"] = a["properties"]["directionUnit"]
        a["speedUnit"] = a["properties"]["speedUnit"]
        a["eventUid"] = a["properties"]["eventUid"]
        a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
        a["counter_direction_bicycleCount"] = a["measures"]["counter_direction_bicycleCount"]
        a["bicycleCount"] = a["measures"]["bicycleCount"]
        a["counter_direction"] = a["measures"]["counter_direction"]
        a["speed"] = a["measures"]["speed"]
        a["direction"] = a["measures"]["direction"]
        san_diego_bicycle_sensor_events_list.append(a)

#     assetEvents_df = pd.DataFrame(assetEvents)
#     assetEvents_df["latitude"] = row.latitude
#     assetEvents_df["longitude"] = row.longitude
#     san_diego_bicycle_sensor_events_list.append(assetEvents)
    
# print(san_diego_bicycle_sensor_events_list)
san_diego_bicycle_sensor_events_df = pd.DataFrame(san_diego_bicycle_sensor_events_list)
san_diego_bicycle_sensor_events_df.dropna(inplace = True)
# print(san_diego_bicycle_sensor_events_df)
    
    
san_diego_bicycle_sensor_events_gdf = gpd.GeoDataFrame(san_diego_bicycle_sensor_events_df, geometry=gpd.points_from_xy(san_diego_bicycle_sensor_events_df.longitude, san_diego_bicycle_sensor_events_df.latitude))
san_diego_bicycle_sensor_events_d = Dataset(san_diego_bicycle_sensor_events_gdf)
san_diego_bicycle_sensor_events_d.upload(table_name='san_diego_bicycle_sensor_events', if_exists='append', credentials=credentials)
